In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.layers import Dense, ReLU, Flatten, LSTM, Input, MaxPooling1D
from tensorflow.keras.layers import Input, Embedding, BatchNormalization, Concatenate
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.models import Sequential, Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from keras.utils import to_categorical
import re
from sklearn.model_selection import train_test_split

Lấy data từ 2 file X_train Y_train X_test và Y_test từ các file X_train_cleaned Y_train_cleaned X_test_cleaned Y_test_cleaned

In [2]:
X_tr=pd.read_csv('Twitter_2/X_train_cleaned.csv',header=None,encoding='latin1')
Y_tr=pd.read_csv('Twitter_2/Y_train_cleaned.csv',header=None)
X_t=pd.read_csv('Twitter_2/X_test_cleaned.csv',header=None,encoding='latin1')
Y_t=pd.read_csv('Twitter_2/Y_test_cleaned.csv',header=None)
X_train = X_tr[0].astype(str).tolist()
X_test = X_t[0].astype(str).tolist()


Chuẩn bị data để nhét vào mô hình chuẩn bị việc train mô hình

In [3]:
encoder=LabelEncoder()
Y_train=encoder.fit_transform(Y_tr)
Y_test=encoder.transform(Y_t)
tokenizer=Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)
sequences=tokenizer.texts_to_sequences(X_train)
tr_x=pad_sequences(sequences,maxlen=50)
tr_y=to_categorical(Y_train)
sequences=tokenizer.texts_to_sequences(X_test)
t_x=pad_sequences(sequences,maxlen=50)
t_y=to_categorical(Y_test)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Tạo mô hình CNN để chạy data

In [4]:
embedding_dim=32
max_len=50
max_word=20000
batchsize=64
epochs=25
def model1():
    inp=Input(shape=(max_len,))

    embed1=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    conv1=Conv1D(filters=100,kernel_size=3,padding='same',activation='relu')(embed1)
    batch1=BatchNormalization()(conv1)
    pool1=GlobalMaxPooling1D()(batch1)

    embed2=Embedding(input_dim=max_word,output_dim=embedding_dim,input_length=max_len)(inp)
    conv2=Conv1D(filters=100,kernel_size=4,padding='same',activation='relu')(embed2)
    batch2=BatchNormalization()(conv2)
    pool2=GlobalMaxPooling1D()(batch2)

    merge=Concatenate(axis=1)([pool1,pool2])
    dense=Dense(128,activation='relu')(merge)
    drop=Dropout(0.3)(dense)
    outp=Dense(3,activation='softmax')(drop)

    model=Model(inputs=inp,outputs=outp)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',Precision,Recall])
    return model

Chạy mô hình với data train

In [5]:
model=model1()
model.summary()
history=model.fit(tr_x, tr_y, epochs=epochs, batch_size=batchsize, validation_split=0.2)

c:\Users\ADMIN.LAPTOP-6P42KB5P\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 32)    │    640,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 32)    │    640,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 50, 100)   │      9,700 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 50, 100)   │     12,900 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 50, 100)   │        400 │ conv1d[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 100)   │        400 │ conv1d_1[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ batch_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 100)       │          0 │ batch_normalizat… │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 200)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     25,728 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        387 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,329,515 (5.07 MB)

 Trainable params: 1,329,115 (5.07 MB)

 Non-trainable params: 400 (1.56 KB)

Epoch 1/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.5110 - loss: 0.5778 - precision: 0.5615 - recall: 0.3824 - val_accuracy: 0.6811 - val_loss: 0.5495 - val_precision: 0.7980 - val_recall: 0.4650
Epoch 2/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.8076 - loss: 0.3044 - precision: 0.8201 - recall: 0.7955 - val_accuracy: 0.7100 - val_loss: 0.4105 - val_precision: 0.7404 - val_recall: 0.6611
Epoch 3/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9181 - loss: 0.1510 - precision: 0.9238 - recall: 0.9128 - val_accuracy: 0.6978 - val_loss: 0.4930 - val_precision: 0.7027 - val_recall: 0.6900
Epoch 4/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9662 - loss: 0.0687 - precision: 0.9681 - recall: 0.9652 - val_accuracy: 0.6922 - val_loss: 0.6407 - val_precision: 0.6947 - val_recall: 0.6891
Epoch 5/25
344/344 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9859 - loss: 0.0329 - precision: 0.9864 - recall: 0.9853 - val_accuracy: 0.6944 - val_los

Thử dữ liệu với data test

In [6]:
pred_label=model.predict(t_x)
predicted=np.argmax(pred_label,axis=1)
true=np.argmax(t_y,axis=1)
print(classification_report(true, predicted))


111/111 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
              precision    recall  f1-score   support

           0       0.62      0.70      0.66      1001
           1       0.63      0.64      0.64      1430
           2       0.79      0.68      0.73      1103

    accuracy                           0.67      3534
   macro avg       0.68      0.67      0.67      3534
weighted avg       0.68      0.67      0.67      3534

